# Introducción a SQL: 2

***
<div class="panel panel-danger">
    <div class='panel-heading'>
    <h4>Antes de empezar</h4>
    </div>
    <div class='panel-body'>
    <p>Para poder ejecutar queries a la base de datos, necesitamos tener instalado antes el paquete [ipython-sql](https://github.com/catherinedevlin/ipython-sql).
    
    <p>Este paquete nos permite escribir las queries sql en las celdas del notebook y visualizar el resultado como tablas html renderizadas
    <p>Ejecuta `!pip install ipython-sql` si es la primera vez que usas este notebook
    </div>
</div>

In [1]:
!pip install ipython-sql

Una vez instalado, podemos invocar el entorno `SQL` usando `%load_ext sql`.
Esto nos permitira usar el magic de ipython `%%sql` al principo de **cada** celda y ejecutar comandos sql directamente

<div class="panel panel-success">
    <div class='panel-heading'>
    <h4>Empecemos</h4>
    </div>
</div>


Los objetivos de este notebook son:



# 1. Creando una base de datos

En la sesión anterior vimos como hacer consultas básicas a una base de datos. En esta sesión veremos como podemos crear y modificar nuestra propia base de datos relacional de forma sencilla.

Las tareas que necesitaremos saber se pueden resumir en:

1. Crear tablas
2. Modificar tablas
3. Eliminar tablas

Para empezar, cargaremos el modulo `sql` que nos permitirá hacer las consultas SQL directamente desde el notebook.

Crear una base de datos `sqlite` es muy sencillo. Para ello, simplemente hay que escribir el comando sql pertinente que es:

`%sql sqlite:///nombredelaBDD.sqlite`

y nos creará el archivo donde podremos crear y modificar tablas.

#### Carga el modulo sql y crea una nueva base de datos sqlite

```python
%load_ext sql
%sql sqlite:///newdb.sqlite

#check table works and is empty
%sql SELECT name FROM sqlite_master WHERE type='table'
```

In [3]:
#copia y pega el snippet anterior
%load_ext sql
%sql sqlite:///newdb.sqlite

#check table works and is empty
%sql SELECT name FROM sqlite_master WHERE type='table'

Done.


name


## Creando tablas

### CREATE TABLE
Para crear una tabla, escribimos:
```SQL
CREATE TABLE table_name ([schema])
```

### INSERT INTO
Para insertar valores en una tabla existente

```SQL
INSERT INTO table_name ([column]) VALUES ([values]);
```

### DROP TABLE
Para eliminar una tabla existente

```SQL
DROP TABLE IF EXISTS table_name
```

**Ojo!:**
 - Cuando trabajamos con cadenas de caracteres 'hola' debemos usar comillas simples.  (SQLite and MySQL no importa, Postgres (aws-redshift) sí!
 - Si no especificamos la PK cuando insertamos una fila, se asignará una PK automáticamente. Si intentamos crear una nueva fila con un PK que ya existe en la tabla, nos devolverá un error.
 - Es una buena práctica preceder la creación de una tabla con la eliminación de la misma, just in case! 


## Schemas
SQL es tipado. Eso implica que cada columna de la tabla debe tener un tipo de datos específico. Al crear la tabla, deberemos especificar el **tipo** y otros atributos. Los tipos y atributos básicos son:

- `INTEGER` (or `INT`): integer-valued numeric data
- `FLOAT`: non-integer numeric data
- `TEXT`: strings or textual data
- `DATE`: a date object

Columns can have optional attributes:

- `PRIMARY KEY`: Especifica la clave primaria.
- `NOT NULL`: Por defecto, los valores de una columna pueden ser nulos. `NOT NULL` obliga a la tabla a que todos los valores de la columna sean especificados.
- `DEFAULT`: specify the default value in this column.

### Ejemplo

Elimina la tabla alumnos si existe y la crea con tres campos: id_alumno, nombre, y edad. El nombre no puede ser nulo mientras que la edad es opcional.

```SQL
DROP TABLE IF EXISTS alumnos;
CREATE TABLE alumnos (
    id_alumno INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    edad INT
);

INSERT INTO alumnos (nombre) VALUES ('Otelo');
INSERT INTO alumnos (nombre, edad) VALUES ('Desdémona', 20);

SELECT * FROM alumnos;
```
Si quisieramos que la edad tuviera un valor por defecto, deberíamos añadir el atributo `DEFAULT` seguido del valor por defecto. 

#### Ejercicio 1: Crea la tabla anterior y haz que la edad tenga por defecto un valor de -1

In [7]:
%%sql

--Hola esto es un comentario
DROP TABLE IF EXISTS alumnos;

CREATE TABLE alumnos (
    id_alumno INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    edad INT DEFAULT -1
);

INSERT INTO alumnos (nombre) VALUES ('Otelo');
INSERT INTO alumnos (nombre, edad) VALUES ('Desdémona', 20);

SELECT * FROM alumnos;

Done.
Done.
1 rows affected.
1 rows affected.
Done.


id_alumno,nombre,edad
1,Otelo,-1
2,Desdémona,20


## Modificar tablas

Existen queries que nos permiten modificar tablas.

```SQL
DELETE FROM table_name [WHERE condition]
```

Elimina las filas de la tabla especificada segun la condición.

```SQL
UPDATE table_name SET column_name1=value [WHERE condition]
```

Actualiza los valores de la columna especificada. Podemos especificar condiciones, por ejemplo, a que identificador lo queremos cambiar o imputar valores nulos de forma persistente.

Si queremos modificar columnas, los comandos son:

```SQL
ALTER TABLE table_name DROP COLUMN column_name (no soportado en sqlite)
```

```SQL
ALTER TABLE table_name ADD COLUMN column_name column_type;
```


#### Ejercicio 2: En la tabla alumnos que has creado en el Ejercicio 1, actualiza la edad de Otelo a 23

In [9]:
%%sql
UPDATE alumnos SET edad=23 
WHERE
    nombre='Otelo'


1 rows affected.


[]

#### Ejercicio 3: En la tabla alumnos, añade la columna `genero` con el valor que corresponde a cada uno de los dos personajes

In [10]:
%%sql

ALTER TABLE alumnos
ADD genero TEXT



Done.


[]

In [14]:
%%sql
UPDATE alumnos SET genero='M' 
WHERE
    nombre LIKE('Ot%');
    
UPDATE alumnos SET genero='F' 
WHERE
    nombre LIKE('%a');
    
SELECT * FROM alumnos;

1 rows affected.
1 rows affected.
Done.


id_alumno,nombre,edad,genero
1,Otelo,23,M
2,Desdémona,20,F


#### Ejercicio 4: En la tabla alumnos, elimina la entrada de Desdemona y añade una nueva

In [21]:
%%sql

DELETE FROM alumnos WHERE nombre LIKE 'Anac%';
INSERT INTO alumnos (nombre, edad, genero) VALUES ('Anacleto', 45,'M');
SELECT * FROM alumnos;

2 rows affected.
1 rows affected.
Done.


id_alumno,nombre,edad,genero
1,Otelo,23,M
2,Anacleto,45,M


#### Ejercicio 5: Haz una query a toda la tabla y guardala en un csv usando pandas

In [22]:
import sqlite3

import pandas as pd

conn = sqlite3.connect('newdb.sqlite')

query = """SELECT * FROM alumnos"""

df = pd.read_sql(query, conn)

In [23]:
df

,id_alumno,nombre,edad,genero
0,1,Otelo,23,M
1,2,Anacleto,45,M


In [24]:
df.to_csv('alunmnos.csv')

In [26]:
!cat alunmnos.csv

,id_alumno,nombre,edad,genero
0,1,Otelo,23,M
1,2,Anacleto,45,M


## Creando tablas SQL: Un atajo con pandas

A estas alturas ya deberíamos estar convenvidos de que pandas es una herramienta increible en muchos aspectos. Uno de ellos es que nos permite crear tablas en bases de datos de forma automática. Si tenemos un pandas.DataFrame() y una conexión a una base de datos, pandas nos creara de forma automática las queries necesarias para crear el esquema y la tabla e insertar todas las filas.

```python
import pandas as pd
import sqlite3

conn = sqlite3.connect('newdb.sqlite')

df = pd.read_csv('somedata.csv')
df.to_sql('table_name', conn, if_exists= "replace", index=False)
```



#### Ejercicio 6: Crea una base de datos nueva `bdd_notas.sqlite`, lee los ficheros csv de la carpeta `data` y crea las tablas estudiantes, asignaturas y notas.

In [ ]:
import sqlite3

import pandas as pd

conn = sqlite3.connect('bdd_notas.sqlite')

df = pd.read_csv('url/alumnos.csv')


df.to_sql('alumnos', conn, if_exist="replace")

# 2. SQL
---

## Query básica

En resumen, la query básica para leer datos de **Una** tabla de nuestra base de datos sería:

```SQL
SELECT expression1, expression2, ...
FROM table_name
[WHERE condition]
[GROUP BY columns]
[HAVING condition]
[ORDER BY columns]
[LIMIT number];
```

---

## Joins

Esta opción nos permite combinar información de distintas tablas y hacer consultas sobre este nuevo conjunto.

### Inner JOIN

Nos devuelve la intersección de las tablas seleccionadas, es decir, aquellas filas donde las distintas columnas tienen valores no nulos.

```SQL
SELECT a.col1,
       a.col2,
       b.col1,
       b.col2
FROM table_a as a
INNER JOIN table_be as b
ON a.key = b.key
```

### LEFT JOIN

El comando LEFT JOIN devuelve todos lor registros de la tabla de la izquierda (LEFT) y los registros correspondientes de la otra tabla, en caso en que no existan devuelve NULL.

```SQL
SELECT a.col1,
       a.col2,
       b.col1,
       b.col2
FROM table_a as a
LEFT JOIN table_be as b
ON a.key = b.key
```



In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('bdd_notas.sqlite')

query = """
SELECT name FROM sqlite_master WHERE type='table'
"""

df = pd.read_sql(query, conn)
df

,name
0,estudiantes
1,asignaturas
2,notas


#### Ejercicio 0: Saca la primera fila con todas las columnas de cada una de las tablas 

In [29]:
query = """ SELECT * FROM estudiantes LIMIT 1"""

pd.read_sql(query, conn)

,student_id,name
0,101,Juan


In [37]:
query = """ SELECT * FROM notas LIMIT 1"""

pd.read_sql(query, conn)

,asign_id,student_id,year,nota
0,201,103,2015,9.2


In [32]:
query = """ SELECT * FROM asignaturas LIMIT 1"""

pd.read_sql(query, conn)

,asign_id,asignatura
0,201,Mates


#### Ejercicio 1: Haz un LEFT JOIN de todas las tablas

In [40]:
query = """

SELECT *
FROM estudiantes as e
LEFT JOIN notas as n ON n.student_id = e.student_id
LEFT JOIN asignaturas as a ON n.asign_id = a.asign_id
"""

pd.read_sql(query, conn)

,student_id,name,asign_id,student_id,year,nota,asign_id,asignatura
0,101,Juan,None,None,NaN,NaN,NaN,None
1,102,Maria,None,None,NaN,NaN,NaN,None
2,103,Pedro,201,103,2015.0,9.2,201.0,Mates
3,103,Pedro,201,103,2016.0,7.3,201.0,Mates
4,103,Pedro,201,103,2017.0,8.6,201.0,Mates
5,103,Pedro,302,103,2015.0,7.3,302.0,Física
6,103,Pedro,302,103,2016.0,4.7,302.0,Física
7,103,Pedro,302,103,2017.0,6.5,302.0,Física
8,103,Pedro,211,103,2015.0,8.5,211.0,Filosofía
9,103,Pedro,211,103,2016.0,8.3,211.0,Filosofía


#### Ejercicio 2: Qué asignatura que tiene peor nota media de todas?

In [42]:
query = """

SELECT a.asignatura, AVG(n.nota) as avg_nota
FROM notas as n
LEFT JOIN asignaturas as a
GROUP BY a.asignatura
ORDER BY avg_nota ASC
LIMIT 1

"""

pd.read_sql(query, conn)

,asignatura,avg_nota
0,Filosofía,6.6


#### Ejercicio 3: Que estudiantes tienen una nota media menor que 5. Agrupalo por años.

In [6]:
query = """ 
SELECT
    e.name, n.year, avg(n.nota) as nota_media
FROM
    notas n
    LEFT JOIN estudiantes e
    ON e.student_id = n.student_id
GROUP BY
    n.year, e.name
HAVING
    AVG(n.nota) < 5

"""

pd.read_sql(query, conn)

,name,year,nota_media
0,Ana,2015,2.0
1,Ana,2016,3.0
2,Susana,2016,4.7


#### Ejercicio 4: Que asignaturas son las peores de cada año? Haz una query para cada año.

In [22]:
# 2015
query = """
SELECT a.asignatura, AVG(n.nota) as avg_nota
FROM notas as n
LEFT JOIN asignaturas as a
ON a.asign_id = n.asign_id
WHERE n.year=2015
GROUP BY a.asignatura
ORDER BY avg_nota ASC
LIMIT 1
"""

pd.read_sql(query, conn)

,asignatura,avg_nota
0,Mates,5.6


In [23]:
# 2016

query = """
SELECT a.asignatura, AVG(n.nota) as avg_nota
FROM notas as n
LEFT JOIN asignaturas as a
ON a.asign_id = n.asign_id
WHERE n.year=2016
GROUP BY a.asignatura
ORDER BY avg_nota ASC
LIMIT 1
"""

pd.read_sql(query, conn)

,asignatura,avg_nota
0,Física,4.7


In [24]:
# 2017

query = """
SELECT a.asignatura, AVG(n.nota) as avg_nota
FROM notas as n
LEFT JOIN asignaturas as a
ON a.asign_id = n.asign_id
WHERE n.year=2017
GROUP BY a.asignatura
ORDER BY avg_nota ASC
LIMIT 1"""

pd.read_sql(query, conn)

,asignatura,avg_nota
0,Física,6.5


### UNION and UNION ALL

Estos comando se usan para unir tablas o resultados de consultas que tienen las mismas columnas.
La diferencia entre `UNION` y `UNION ALL` es que la primera elimina los registros duplicados mientras que la última no.

Cómo puedes imaginar, estos comandos se pueden usar para filtrar y agregar.

La sintaxis básica es:

```SQL
(SELECT statement)
 UNION [ALL]
(SELECT STATEMENT)```


Recuerda que las SELECT deben tener el mismo número de columnas.

#### Ejercicio 4.b: Realiza la misma consulta que en el Ejercicio 4 pero en una sola query usando UNION ALL

In [43]:
query = """

SELECT a.asignatura,n.year, AVG(n.nota) as avg_nota
FROM notas as n
LEFT JOIN asignaturas as a
ON a.asign_id = n.asign_id
WHERE n.year=2015
GROUP BY a.asignatura

UNION ALL

SELECT a.asignatura,n.year, AVG(n.nota) as avg_nota
FROM notas as n
LEFT JOIN asignaturas as a
ON a.asign_id = n.asign_id
WHERE n.year=2016

UNION ALL

SELECT a.asignatura, n.year, AVG(n.nota) as avg_nota
FROM notas as n
LEFT JOIN asignaturas as a
ON a.asign_id = n.asign_id
WHERE n.year = 2017

GROUP BY a.asignatura
ORDER BY avg_nota asc
"""

pd.read_sql(query, conn)

,asignatura,year,avg_nota
0,Mates,2015,5.6
1,Física,2016,5.6
2,Física,2017,6.5
3,Filosofía,2017,7.2
4,Física,2015,7.3
5,Mates,2017,7.3
6,Filosofía,2015,8.5


Imagina que tenemos 15 años distintos. Sería una tarea un tanto repetitiva la de picar a mano una query que nos devolviera lo mismo la peor asignatura de cada año.

Una de las ventajas de integrar el flujo de SQL en python (pandas) es que podemos hacer este tipo de tareas de forma programática.

Vamos a intentar automatizar el proceso. De esta forma, si hay alguna nueva entrada en la tabla o base de datos, al volver a ejecutar este trozo tendremos el resultado actualizado

In [45]:
# Haz una query que te devuelva todos los años distintos de la tabla notas

query_years = """SELECT DISTINCT(year) FROM notas ORDER BY year ASC"""
years_unique = pd.read_sql(query_years, conn)

#1. Inicializa una lista vacia
list_df = []
#2. Haz un barrido para todas los años en la columna year de years_unique.
#     2.a Escribe una query donde le pases como variable el año. (Tip: "Hola {j}".foógicarmat(j='juan'))
#     2.b Realiza una consulta a la BDD con la query y guarda el resultado en un df
#     2.c Guarda el resultado en el último puesto de la lista (Tip: append())
for year in years_unique['year'].values:
    query = """
            SELECT a.asignatura, AVG(n.nota) as avg_nota
            FROM notas as n
            LEFT JOIN asignaturas as a
            ON a.asign_id = n.asign_id
            WHERE n.year={y}
            GROUP BY a.asignatura
            ORDER BY avg_nota ASC
            LIMIT 1""".format(y=year)
    df = pd.read_sql(query, conn)
    list_df.append(df)


#3. Haz una concatenación de los df en la lista (TIP: pd.concat())
df_all = pd.concat(list_df)

#4. Haz un print del resultado

In [46]:
df_all

,asignatura,avg_nota
0,Mates,5.6
0,Física,4.7
0,Física,6.5


## Subqueries

En el ejemplo anterior, podríamos haber decidido que en vez de filtrar año a año y unir las tablas lo que nos resulta más eficiente es primero realizar una agregación por año y asignatura para después realizar un filtrado del resultado y quedarnos con el menor.

SQL permite realizar este tipo de acciones mediante el anidado de queries. Es decir, sustituir alguno de los campos en la query por otra query que nos haga un primer filtrado.

Por ejemplo,

```SQL

SELECT nota
FROM notas
WHERE asign_id = (SELECT asign_id
                  FROM asignaturas
                  WHERE asignatura='Mates')
```

Nos devolvería las notas de la asignatura de matemáticas.

#### Ejercicio 5: Encuentra la peor asignatura de cada año:
1. 5a: Realiza una query que devuelva la media de cada asignatura por año y comprueba que funciona
1. 5b: Usa la consulta del punto anterior como tabla en el FROM y devuelve la menor de las notas de cada año

In [52]:
query = """

select year, asignatura, avg(nota) as avg_nota
from asignaturas as a
inner join notas as n on n.asign_id = a.asign_id
group by year, asignatura

"""

pd.read_sql(query, conn)

,year,asignatura,avg_nota
0,2015,Filosofía,8.50
1,2015,Física,7.30
2,2015,Mates,5.60
3,2016,Filosofía,8.30
4,2016,Física,4.70
5,2016,Mates,5.15
6,2017,Filosofía,7.20
7,2017,Física,6.50
8,2017,Mates,7.30


In [53]:



query = """

SELECT avg_note_year.year, avg_note_year.asignatura, min(avg_note_year.avg_nota)
FROM (

    select year, asignatura, avg(nota) as avg_nota
    from asignaturas as a
    inner join notas as n on n.asign_id = a.asign_id
    group by year, asignatura) as avg_note_year

GROUP BYyear

"""

pd.read_sql(query, conn)

,year,asignatura,min(avg_nota)
0,2015,Mates,5.6
1,2016,Física,4.7
2,2017,Física,6.5


#### Ejercicio 6: Que estudiantes no se han presentado a ningún examen?

#### Ejercicio 7: Quienes son los mejores estudiantes de cada asignatura de cada año?